# Feature engineering

## Introduction
We always look at a round from the point of view of team_1.

## Features: 
- **target**: team_1 won the round
- **kills**: Amount of players killed by team_1
- **deaths**: Amount of players that died from team_1
- **equipment_value_ratio**: Total equipment value compared to equipment value of team_2. *This is better than saving the equipment values from both the teams. $650 in a pistol round is normal. $650 in round 10 can probably be considered as a saving round.*
- **first_kill**: Did team_1 make the first kill. Making the first kill results in a higher chance of winning the round.
- **map**: maybe there are more clutches on Mirage?
- **damage_done**: Amount of damage team1 did to team2
- **damage_taken**: Amount of damage team2 did to team1
- **median_player_health**: Median health of all players of the team.
- **is_bomb_planted**:

- **utility**: [TODO]

# Features to implement
- **defuse_kit_count**: Amount of defusekits.
  - **values**: [None,0,1,2,3,4,5] (None when team_1 are terrorists)
- **alive_ratio_on_bomb_planted**: Amount of players alive from team_1 compared to players alive from team_2 when the bomb is planted.
- **distance_from_bomb_on_plant**: Average distance from team_1 when bomb is planted. (Maybe they are already saving)




In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd



In [11]:
# Feature engineering
from feature_engineering import db
from feature_engineering import parser
from dataclasses import make_dataclass,fields
from bson.objectid import ObjectId
from sklearn.model_selection import train_test_split
# from sklearn.preprocessing import minmax_scale

collections = db.get_collections()
Round = make_dataclass("Round", [("kills", int),("deaths", int),("first_blood", int),("round_winstreak", int)])

parsed_rounds = []
target = []
# TODO: remove this, we limit to 50 while testing.
matches = collections["matches"].find({}).limit(50)

# MATCHES
for match in matches:
  round_winstreak = 0
  team_1_id = match["teams"][0]

  # ROUND
  for round_id in match["rounds"]:

    # Sometimes a round is not parsed correctly and the data is not available.

    round = collections["rounds"].find_one({ "_id": ObjectId(round_id)})
    if round is None:
      continue

    # Create a set of 4 ticks: @25%,@50%,@75%,@100%.
    # To learn the model it does not always has all kills and all information.
    duration = round["endtick"] - round["starttick"], 
    ticks = map(lambda n: round["starttick"] + duration*n,[.35,.5,.75,1])

    for tick in ticks:
      round = parser.Round(round_id,team_1_id, tick)
      (kills, deaths) = round.kills_and_deaths()

      if round.is_win() :
        target.append(1)
        round_winstreak+= 1
      else:
        target.append(0)
        round_winstreak=0

      parsed_rounds.append(Round(kills=kills,deaths=deaths,first_blood=round.is_first_blood(), round_winstreak = round_winstreak))

print("Parsed rounds:", len(parsed_rounds))
data = pd.DataFrame(parsed_rounds)



/home/emiel/.cache/pypoetry/virtualenvs/feature-engineering-i-Acv69h-py3.8/lib/python3.8/site-packages/pymongo/common.py:801: UserWarning: Unknown option 3t.uriversion
  warnings.warn(str(exc))
/home/emiel/.cache/pypoetry/virtualenvs/feature-engineering-i-Acv69h-py3.8/lib/python3.8/site-packages/pymongo/common.py:801: UserWarning: Unknown option 3t.connection.name
  warnings.warn(str(exc))
/home/emiel/.cache/pypoetry/virtualenvs/feature-engineering-i-Acv69h-py3.8/lib/python3.8/site-packages/pymongo/common.py:801: UserWarning: Unknown option 3t.alwaysshowauthdb
  warnings.warn(str(exc))
/home/emiel/.cache/pypoetry/virtualenvs/feature-engineering-i-Acv69h-py3.8/lib/python3.8/site-packages/pymongo/common.py:801: UserWarning: Unknown option 3t.alwaysshowdbfromuserrole
  warnings.warn(str(exc))


Parsed rounds: 1336


In [15]:
# We should how diverse our dataset is. If in 80% of our games team_1 wins.
# We will learn the model that it can always pick team_1 and get an accuracy of 80%. We don't want that :)

# Split the data in training and testing data
train_X,test_X, train_Y, test_Y = train_test_split(data,target,test_size=0.2, random_state=42)




      kills  deaths  first_blood  round_winstreak
1112      2       3            0                0
966       5       4            1                0
598       3       5            0                0
170       5       1            0                0
275       2       4            1                4
...     ...     ...          ...              ...
1095      5       2            1                0
1130      5       4            1                2
1294      1       5            0                2
860       2       5            0                0
1126      3       5            1                0

[1068 rows x 4 columns]


# Hyperparameter tuning
With SVM we try to find the maximum margin separator between our two classes (team_1_wins, team_2_wins). This is the line the furtest from the nearest training data points. SVM calculates the distance to the closest datapoint for each possible line and picks the one with the highest distance. This makes an SVM a **maximum margin estimator. 

In most real probelms, it is not possible to find the perfect separating plane. Sometimes there are datapoints that are not closer to the other class. To handle this we allow the SVM to soften the margin. Which means we allow some of the points to creep into the margin if that allows a better fit. This transforms our SVM into a soft-margin classifier since we are allowing for a few mistakes. This is typically called C.

C is a hyperparameter that needs to be tuned based on the dataset.

In [13]:
from sklearn import svm
from sklearn.model_selection import GridSearchCV

parameters = { "kernel": ["rbf", "linear", "sigmoid"], "C": range(1,50)}
clf = GridSearchCV(svm.SVC(), parameters)
clf.fit(train_X, train_Y)

print(clf.cv_results_)

# TODO: handle overfitting, with cross validation and folding

{'mean_fit_time': array([0.00645862, 0.00208349, 0.01907997, 0.00500402, 0.00154786,
       0.01751208, 0.00468102, 0.00144796, 0.01682172, 0.00413818,
       0.00156727, 0.01651382, 0.00414653, 0.00138545, 0.01678123,
       0.00360975, 0.00149598, 0.01642499, 0.00364847, 0.0013895 ,
       0.01647835, 0.00376282, 0.00162473, 0.01673584, 0.00381927,
       0.00136185, 0.01648011, 0.00380902, 0.0013968 , 0.01660876,
       0.00356507, 0.00134926, 0.01643367, 0.00358562, 0.00138903,
       0.01654758, 0.00370941, 0.0015862 , 0.01626782, 0.00377817,
       0.00144601, 0.01674576, 0.00389314, 0.00143971, 0.01679482,
       0.0040482 , 0.0016171 , 0.01676817, 0.00400162, 0.00183597,
       0.01738992, 0.0045608 , 0.00143352, 0.01725535, 0.00413218,
       0.00151873, 0.01684837, 0.00390897, 0.00153913, 0.01705256,
       0.00447259, 0.00147524, 0.0167932 , 0.00402122, 0.00139136,
       0.01786785, 0.00417509, 0.00172329, 0.01664643, 0.00440483,
       0.00133829, 0.01734829, 0.00437651, 0

In [14]:
from sklearn.metrics import accuracy_score, confusion_matrix
# Performance metrics

prediction = clf.predict(test_X)
accuracy = accuracy_score(test_Y, prediction)
print(accuracy)

confusion_matrix(test_Y, prediction)


1.0


array([[130,   0],
       [  0, 138]])

# Live demo parsing

A second part, we unfortunately had no time to implement, is **live demo parsing**.

We need to start by creating a probability N by N matrix, with N the different states.
This is also called a ** stochastic matrix**. Each combination of a row(x) and a column(y) is the probability to go from stateX to stateY.

Since we eventually want to find the winner of a game we also need something called absorbing states. **An absorbing state is a state that, once entered, cannot be left**
